Воспользовался уже готовой арзитектурой и просто обучил ее. Была так же идея сделать более умную аугментацию, а не только флиппать иззображения. Нашел готовую библиотеку imgaug, в которой можно реалтзовывать pipeline, но он постоянно падал с исключениями и я не смог отдебажить. Но использовав AffineTransformation из OpenCV на другой архитектуре мне удалось увеличить accuracy на 10%.

In [ ]:
from __future__ import print_function
from lasagne import init
from lasagne.layers import Conv2DLayer as ConvLayer
from lasagne.layers import ElemwiseSumLayer
from lasagne.layers import InputLayer
from lasagne.layers import DenseLayer
from lasagne.layers import GlobalPoolLayer
from lasagne.layers import PadLayer
from lasagne.layers import ExpressionLayer
from lasagne.layers import NonlinearityLayer
from lasagne.nonlinearities import softmax, rectify
from lasagne.layers import batch_norm
import lasagne.nonlinearities
import sys
import os
import time
import string
import random
import pickle

import numpy as np
import theano
import theano.tensor as T
import lasagne

def unpickle(file):
    import cPickle
    fo = open(file, 'rb')
    dict = cPickle.load(fo)
    fo.close()
    return dict

from lasagne.layers import Conv2DLayer as ConvLayer
from lasagne.layers import ElemwiseSumLayer
from lasagne.layers import InputLayer
from lasagne.layers import DenseLayer
from lasagne.layers import GlobalPoolLayer
from lasagne.layers import PadLayer
from lasagne.layers import ExpressionLayer
from lasagne.layers import NonlinearityLayer
from lasagne.nonlinearities import softmax, rectify
from lasagne.layers import batch_norm

def build_cnn(input_var=None, n=5):
    
    # create a residual learning building block with two stacked 3x3 convlayers as in paper
    def residual_block(l, increase_dim=False, projection=False):
        input_num_filters = l.output_shape[1]
        if increase_dim:
            first_stride = (2,2)
            out_num_filters = input_num_filters*2
        else:
            first_stride = (1,1)
            out_num_filters = input_num_filters

        stack_1 = batch_norm(ConvLayer(l, num_filters=out_num_filters, filter_size=(3,3), stride=first_stride, nonlinearity=rectify, pad='same', W=lasagne.init.HeNormal(gain='relu'), flip_filters=False))
        stack_2 = batch_norm(ConvLayer(stack_1, num_filters=out_num_filters, filter_size=(3,3), stride=(1,1), nonlinearity=None, pad='same', W=lasagne.init.HeNormal(gain='relu'), flip_filters=False))
        
        # add shortcut connections
        if increase_dim:
            if projection:
                # projection shortcut, as option B in paper
                projection = batch_norm(ConvLayer(l, num_filters=out_num_filters, filter_size=(1,1), stride=(2,2), nonlinearity=None, pad='same', b=None, flip_filters=False))
                block = NonlinearityLayer(ElemwiseSumLayer([stack_2, projection]),nonlinearity=rectify)
            else:
                # identity shortcut, as option A in paper
                identity = ExpressionLayer(l, lambda X: X[:, :, ::2, ::2], lambda s: (s[0], s[1], s[2]//2, s[3]//2))
                padding = PadLayer(identity, [out_num_filters//4,0,0], batch_ndim=1)
                block = NonlinearityLayer(ElemwiseSumLayer([stack_2, padding]),nonlinearity=rectify)
        else:
            block = NonlinearityLayer(ElemwiseSumLayer([stack_2, l]),nonlinearity=rectify)
        
        return block

    # Building the network
    l_in = InputLayer(shape=(None, 3, 32, 32), input_var=input_var)

    # first layer, output is 16 x 32 x 32
    l = batch_norm(ConvLayer(l_in, num_filters=16, filter_size=(3,3), stride=(1,1), nonlinearity=rectify, pad='same', W=lasagne.init.HeNormal(gain='relu'), flip_filters=False))
    
    # first stack of residual blocks, output is 16 x 32 x 32
    for _ in range(n):
        l = residual_block(l)

    # second stack of residual blocks, output is 32 x 16 x 16
    l = residual_block(l, increase_dim=True)
    for _ in range(1,n):
        l = residual_block(l)

    # third stack of residual blocks, output is 64 x 8 x 8
    l = residual_block(l, increase_dim=True)
    for _ in range(1,n):
        l = residual_block(l)
    
    # average pooling
    l = GlobalPoolLayer(l)

    # fully connected layer
    network = DenseLayer(
            l, num_units=10,
            W=lasagne.init.HeNormal(),
            nonlinearity=softmax)

    return network

# ############################# Batch iterator ###############################

def iterate_minibatches(inputs, targets, batchsize, shuffle=False, augment=False):
    assert len(inputs) == len(targets)
    if shuffle:
        indices = np.arange(len(inputs))
        np.random.shuffle(indices)
    for start_idx in range(0, len(inputs) - batchsize + 1, batchsize):
        if shuffle:
            excerpt = indices[start_idx:start_idx + batchsize]
        else:
            excerpt = slice(start_idx, start_idx + batchsize)
        if augment:
            # as in paper : 
            # pad feature arrays with 4 pixels on each side
            # and do random cropping of 32x32
            padded = np.pad(inputs[excerpt],((0,0),(0,0),(4,4),(4,4)),mode='constant')
            random_cropped = np.zeros(inputs[excerpt].shape, dtype=np.float32)
            crops = np.random.random_integers(0,high=8,size=(batchsize,2))
            for r in range(batchsize):
                random_cropped[r,:,:,:] = padded[r,:,crops[r,0]:(crops[r,0]+32),crops[r,1]:(crops[r,1]+32)]
            inp_exc = random_cropped
        else:
            inp_exc = inputs[excerpt]

        yield inp_exc, targets[excerpt]

n=5
num_epochs=82
model=None

# Load the dataset
xs = []
ys = []
for j in range(5):
    d = unpickle('cifar-10-batches-py/data_batch_'+`j+1`)
    x = d['data']
    y = d['labels']
    xs.append(x)
    ys.append(y)

d = unpickle('cifar-10-batches-py/test_batch')
xs.append(d['data'])
ys.append(d['labels'])

x = np.concatenate(xs)/np.float32(255)
y = np.concatenate(ys)
x = np.dstack((x[:, :1024], x[:, 1024:2048], x[:, 2048:]))
x = x.reshape((x.shape[0], 32, 32, 3)).transpose(0,3,1,2)

# subtract per-pixel mean
pixel_mean = np.mean(x[0:50000],axis=0)
x -= pixel_mean

# create mirrored images
X_train = x[0:50000,:,:,:]
Y_train = y[0:50000]
X_train_flip = X_train[:,:,:,::-1]
Y_train_flip = Y_train
X_train = np.concatenate((X_train,X_train_flip),axis=0)
Y_train = np.concatenate((Y_train,Y_train_flip),axis=0)

X_test = x[50000:,:,:,:]
Y_test = y[50000:]

data = dict(
        X_train=lasagne.utils.floatX(X_train),
        Y_train=Y_train.astype('int32'),
        X_test = lasagne.utils.floatX(X_test),
        Y_test = Y_test.astype('int32'),)

X_train = data['X_train']
Y_train = data['Y_train']
X_test = data['X_test']
Y_test = data['Y_test']

input_var = T.tensor4('inputs')
target_var = T.ivector('targets')

network = build_cnn(input_var, n)

In [ ]:
prediction = lasagne.layers.get_output(network)
loss = lasagne.objectives.categorical_crossentropy(prediction, target_var)
loss = loss.mean()
all_layers = lasagne.layers.get_all_layers(network)
l2_penalty = lasagne.regularization.regularize_layer_params(all_layers, lasagne.regularization.l2) * 0.0001
loss = loss + l2_penalty

params = lasagne.layers.get_all_params(network, trainable=True)
lr = 0.1
sh_lr = theano.shared(lasagne.utils.floatX(lr))
updates = lasagne.updates.momentum(
        loss, params, learning_rate=sh_lr, momentum=0.9)


train_fn = theano.function([input_var, target_var], loss, updates=updates)

test_prediction = lasagne.layers.get_output(network, deterministic=True)
test_loss = lasagne.objectives.categorical_crossentropy(test_prediction,
                                                        target_var)
test_loss = test_loss.mean()
test_acc = T.mean(T.eq(T.argmax(test_prediction, axis=1), target_var),
                  dtype=theano.config.floatX)

val_fn = theano.function([input_var, target_var], [test_loss, test_acc])

Подготовили все и сделали структуру для вычислений, начнем теперь обучать сеть. В статье предлагается повторять по 5 раз блоки, и обучали они ее на 82 эпохах

In [ ]:
for epoch in range(num_epochs):
    train_indices = np.arange(100000)
    np.random.shuffle(train_indices)
    X_train = X_train[train_indices,:,:,:]
    Y_train = Y_train[train_indices]

    train_err = 0
    train_batches = 0
    start_time = time.time()
    for batch in iterate_minibatches(X_train, Y_train, 128, shuffle=True, augment=True):
        inputs, targets = batch
        train_err += train_fn(inputs, targets)
        train_batches += 1

    val_err = 0
    val_acc = 0
    val_batches = 0
    for batch in iterate_minibatches(X_test, Y_test, 500, shuffle=False):
        inputs, targets = batch
        err, acc = val_fn(inputs, targets)
        val_err += err
        val_acc += acc
        val_batches += 1

    if (epoch+1) == 41 or (epoch+1) == 61:
        new_lr = sh_lr.get_value() * 0.1
        sh_lr.set_value(lasagne.utils.floatX(new_lr))

In [4]:
# np.savez('cifar10_deep_residual_model.npz', *lasagne.layers.get_all_param_values(network))

Сеть довольно быстро обуалась, уже на 4 эпохе на тесте была точность около 80%

In [3]:
# Calculate validation error of model:
test_err = 0
test_acc = 0
test_batches = 0
for batch in iterate_minibatches(X_test, Y_test, 500, shuffle=False):
    inputs, targets = batch
    err, acc = val_fn(inputs, targets)
    test_err += err
    test_acc += acc
    test_batches += 1

Теперь посмотрим на качество на тесте.

In [10]:
print("Final results:")
print("  test loss:\t\t\t{:.6f}".format(test_err / test_batches))
print("  test accuracy:\t\t{:.2f} %".format(
    test_acc / test_batches * 100))

Final results:
  test loss:			0.326712
  test accuracy:		92.65 %


In [ ]:
!mkdir cifar10
# # !curl -o cifar-10-python.tar.gz https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz
!tar -xvzf cifar-10-python.tar.gz -C cifar10

In [ ]:
import matplotlib
matplotlib.use('Agg')

import os
import numpy as np
import pandas as pd

%matplotlib inline

In [ ]:
from sklearn.cross_validation import StratifiedKFold
from sklearn.preprocessing import LabelEncoder


In [1]:
from lasagne import init
from lasagne.layers import Conv2DLayer as ConvLayer
#from lasagne.layers.dnn import Conv2DDNNLayer as ConvLayer
from lasagne.layers import ElemwiseSumLayer
from lasagne.layers import InputLayer
from lasagne.layers import DenseLayer
from lasagne.layers import GlobalPoolLayer
from lasagne.layers import PadLayer
from lasagne.layers import ExpressionLayer
from lasagne.layers import NonlinearityLayer
from lasagne.nonlinearities import softmax, rectify
from lasagne.layers import batch_norm
import lasagne.nonlinearities

Using cuDNN version 5110 on context None
Mapped name None to device cuda: GeForce GTX 1080 (0000:08:00.0)


In [ ]:
# from cifar import load_CIFAR10
# plt.rcParams['figure.figsize'] = (10.0, 8.0) 

# cifar10_dir = './cifar10/cifar-10-batches-py'
# X_train, y_train, X_test, y_test = load_CIFAR10(cifar10_dir)

<h1 align="center">First of all -- Checking Questions</h1> 

**Вопрос 1**: Чем отличаются современные сверточные сети от сетей 5 летней давности?

Раньше сети имели более простую структуру, на данный момент сети имеют огроиное число слоев и другую структуру иногда, например, из-за проблем, что далеко от конца градиент при бэкпропе слабо доходит и делаются оутпуты не только 1 слой

**Вопрос 2**: Какие неприятности могут возникнуть во время обучения современных нейросетей?

Слишком много параметров для оптимизации, много слоев , где затухать градиент может.

**Вопрос 3**: У вас есть очень маленький датасет из 100 картинок, классификация, но вы очень хотите использовать нейросеть, какие неприятности вас ждут и как их решить? что делать если первый вариант  решения не заработает?

Это очень мало данных для обучения, если мы дадим даже искаженную на чуть-чуть или сдвинутую картинку, то скорее всего сеть не сможет нормально классифицировать, она будет искать именно такое изображение и переобцчится


**Вопрос 4**: Как сделать стайл трансфер для музыки? oO

Строится спектрограмма, там используется оконное преобразование Фурье и уже на спектрограмму накладывают стиль как и на картинку, только там по-другому нужно свертку делать

In [ ]:
import lasagne
from theano import tensor as T
from lasagne.nonlinearities import *

# input_X = T.tensor4("X")
# target_y = T.vector("target Y integer",dtype='int32')


### Соберите нейронку: 
- Many times x (Conv+Pool)
- Many small convolutions like 3x3
- Batch Norm 
- Residual Connection
- Data Augmentation 
- Learning rate Schedule 
- ...

### Для вдохновения 
- http://torch.ch/blog/2015/07/30/cifar.html
- https://github.com/szagoruyko/wide-residual-networks 

### Самое интересное
- Для сдачи задания нужно набрать на точность тесте > **92.5**% (это займет много времени, торопитесь :) )
- Для получения бонусных баллов > **95.0**%
- Будет очень хорошо если вы придумаете свою архитектуру или сможете обучить что-то из вышеперечисленного :)
- А для обучения всего этого добра вам будет куда удобнее использовать GPU на Amazon 
    - Инструкция https://github.com/persiyanov/ml-mipt/tree/master/amazon-howto 
    - Вам помогут tmux, CuDNN, ssh tunnel, nvidia-smi, ... 
    - Have fun :)

In [ ]:
# from lasagne.layers.dnn import Conv2DDNNLayer, MaxPool2DDNNLayer
from nolearn.lasagne import NeuralNet
from lasagne.layers import Conv2DLayer, MaxPool2DLayer, FeaturePoolLayer, DropoutLayer

Пытался использовать другую архитектуру, но она не смогла дать не нужный резулттат

In [ ]:
net = NeuralNet(
    layers=[
        (InputLayer, dict(name='in', shape=(None, 3, image_size, image_size))),

        (Conv2DLayer, dict(name='l1c1', num_filters=32, filter_size=(3, 3), pad='same', W=init.Orthogonal(2**.5))),
        (Conv2DLayer, dict(name='l1c2', num_filters=32, filter_size=(3, 3), pad='same', W=init.Orthogonal(2**.5))),
        (Conv2DLayer, dict(name='l1c3', num_filters=32, filter_size=(3, 3), pad='same', W=init.Orthogonal(2**.5))),
        (Conv2DLayer, dict(name='l1c4', num_filters=48, filter_size=(3, 3), pad='same', W=init.Orthogonal(2**.5))),
        (Conv2DLayer, dict(name='l1c5', num_filters=48, filter_size=(3, 3), pad='same', W=init.Orthogonal(2**.5))),
        (MaxPool2DLayer, dict(name='l1p', pool_size=(2,2), stride=(1,1))),

        (Conv2DLayer, dict(name='l2c1', num_filters=80, filter_size=(3, 3), pad='same', W=init.Orthogonal(2**.5))),
        (Conv2DLayer, dict(name='l2c2', num_filters=80, filter_size=(3, 3), pad='same', W=init.Orthogonal(2**.5))),
        (Conv2DLayer, dict(name='l2c3', num_filters=80, filter_size=(3, 3), pad='same', W=init.Orthogonal(2**.5))),
        (Conv2DLayer, dict(name='l2c4', num_filters=80, filter_size=(3, 3), pad='same', W=init.Orthogonal(2**.5))),
        (Conv2DLayer, dict(name='l2c5', num_filters=80, filter_size=(3, 3), pad='same', W=init.Orthogonal(2**.5))),
        (MaxPool2DLayer, dict(name='l2p', pool_size=(2,2), stride=(1,1))),
        
        (Conv2DLayer, dict(name='l3c1', num_filters=128, filter_size=(3, 3), pad='same', W=init.Orthogonal(2**.5))),
        (Conv2DLayer, dict(name='l3c2', num_filters=128, filter_size=(3, 3), pad='same', W=init.Orthogonal(2**.5))),
        (Conv2DLayer, dict(name='l3c3', num_filters=128, filter_size=(3, 3), pad='same', W=init.Orthogonal(2**.5))),
        (Conv2DLayer, dict(name='l3c4', num_filters=128, filter_size=(3, 3), pad='same', W=init.Orthogonal(2**.5))),
        (Conv2DLayer, dict(name='l3c5', num_filters=128, filter_size=(3, 3), pad='same', W=init.Orthogonal(2**.5))),
        (MaxPool2DLayer, dict(name='l3p', pool_size=(8,8), stride=(1,1))),

        (DenseLayer, dict(name='l4', num_units=1000)),

        (DenseLayer, dict(name='out', num_units=n_classes, nonlinearity=lasagne.nonlinearities.softmax)),
    ],
    
    regression=False,
    objective_loss_function=lasagne.objectives.categorical_crossentropy,

    update=lasagne.updates.adam,

    batch_iterator_train=train_iterator,
    batch_iterator_test=test_iterator,

    on_epoch_finished=[
        save_weights,
        save_training_history,
        plot_training_history
    ],

    verbose=10,
    max_epochs=250,
)

### Вот и всё, пошли её учить

## Заполните форму

https://goo.gl/forms/EeadABISlVmdJqgr2 